# Scraping beritajatim.com

## Import Library

**requests** : digunakan untuk membuat permintaan HTTP ke situs web

**BeautifulSoup** : digunakan untuk mengurai konten HTML



In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Scraping

Fungsi ini melakukan iterasi melalui setiap halaman berita dalam rentang yang ditentukan oleh hal.
Pada setiap halaman, fungsi mengambil konten HTML menggunakan requests.get() dari tautan yang dibentuk berdasarkan nomor halaman.
Jika respons dari permintaan adalah 200 (artinya halaman berhasil dimuat), konten halaman di-parse menggunakan BeautifulSoup untuk ekstraksi informasi.
Fungsi mencari blok konten berita dengan class widget berita-list.
Di dalam blok konten tersebut, ia mencari semua elemen berita dengan class berita.
Untuk setiap artikel berita, fungsi mengekstrak judul dari elemen h3 dan tautan dari elemen a di dalamnya.
Selanjutnya, fungsi melakukan permintaan HTTP GET ke tautan artikel untuk mengambil konten dari setiap artikel secara individu.
Jika konten artikel berhasil diambil (status respons 200), fungsi mengekstrak konten dari elemen dengan class entry-content dan mengumpulkannya.
Setelah mengumpulkan judul, konten, dan kategori dari setiap artikel, informasi tersebut dimasukkan ke dalam list data.
Akhirnya, fungsi mengembalikan list data yang berisi informasi yang telah berhasil di-scrape dari halaman-halaman berita.










In [2]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup

async def fetch_content(session, url):
    async with session.get(url) as response:
        return await response.text()

async def scrape_beritajatim(link, hal, kategori):
    category = kategori
    data = []
    base_url = link
    async with aiohttp.ClientSession() as session:
        tasks = []
        for page in range(1, hal + 1):
            url = f'{base_url}page/{page}/'
            tasks.append(fetch_content(session, url))

        pages_content = await asyncio.gather(*tasks)
        for page_content in pages_content:
            soup = BeautifulSoup(page_content, 'html.parser')
            blok = soup.find('main', class_='site-main')
            articles = blok.find_all('div', class_='berita')
            for article in articles:
                link = article.find('a')['href']
                title = article.find('h3').text.strip()
                content_response = await fetch_content(session, link)
                print(title)
                if content_response:
                    content_soup = BeautifulSoup(content_response, "html.parser")
                    _class_content = content_soup.find('div', class_='entry-content')
                    tanggal = content_soup.find('time')  # Mengambil elemen tanggal
                    if _class_content and tanggal:
                        content_text = ""
                        for paragraph in _class_content.find_all(['p']):
                            content_text += paragraph.get_text(strip=True).replace('\n', '').replace('ADVERTISEMENT','').replace('SCROLL TO RESUME CONTENT','')
                        tanggal_publish = tanggal.get_text(strip=True)  # Mendapatkan tanggal publikasi
                        data.append([title, content_text, tanggal_publish, category])

    return data



Scraping dilakukan perkategori
Hasil scraping perkategori kemudian dirubah menjadi dataframe dan digabungkan.
Data hasil penggabungan kemudian dilakukan random agar datanya tidak tersusun urut berdasarkan jenis kategori.
Kemudian data disimpan.






In [3]:
pendidikan = await scrape_beritajatim('https://beritajatim.com/kanal/pendidikan-kesehatan/',40,'Pendidikan')

5 PTN di Indonesia dengan Kelas Internasional
PPK Ormawa HMPBK Unesa Boyong Tiga Penghargaan Abdidaya Ormawa
Rektor Unitomo Ajak Mahasiswanya Tak Apatis pada Politik
PWI Gresik dan SMA Sunan Giri Gelar Pelatihan Jurnalistik
Dialog Gusdurian di Untag Surabaya Ajak Perluas Demokrasi
Seleksi Masuk PTN 2023, Begini Tanggapan UB dan Polinema
Polinema Kukuhkan 5 Guru Besar Berbagai Bidang Ilmu
Usaha Yankes Terbaik, Ponorogo Buka Labkesda Tambakbayan
2024 Mendatang, Universitas Ma Chung Kukuhkan Dua Profesor Baru Bidang TI dan Sastra Inggris
Gelar 2nd EBESSIC, FEB Unisma Datangkan Peneliti dan Praktisi dari 13 Negara
Pusdikbud UGM Launching Permainan Edukasi Truwelu
Kasus DBD Tidak Tinggi, Alasan Nyamuk Wolbachia Tidak Diterapkan di Surabaya
Kejari Tuban Ajak Pelajar SMA Melek Bahaya Korupsi
4 Kali Juara Umum MTMQ-MN, UM Optimis Terus Unggul di 2025
FEB Unisma Bahas Ekonomi Bareng Peneliti dari 13 Negara
Periksa Kesehatan Pakai PeriksaDulu.com Bisa Gunakan BPJS
Fikom Universitas Ciputra Surab

In [4]:
sport = await scrape_beritajatim('https://beritajatim.com/kanal/olahraga/', 40, 'Sport')

Kurang Persiapan Jadi Alasan Kekalahan Perseba Bangkalan
Kantongi Poin Penuh Kontra Perseba, Ini ‘PR’ Benahi Pemain Perssu
Gagal Raih Kemenangan, Karteker Persebaya Enggan Kasih Komentar
Satu Poin untuk Persis Solo, Awal Kebangkitan Selamatkan Tim di Klasemen
Leg Kedua, Perssu MC Tundukkan Perseba Bangkalan 2-1
Kalah 1-0 Atas Persenga Nganjuk, Pelatih PSMP Mojokerto Kecewa
Gol Cepat Persenga Nganjuk Jadi Petaka Bagi PSMP Mojokerto
Gol Cepat Qusyairi Lubis Sukses Bawa Persepam Raih 3 Poin dari Persesa
Babak Pertama, Perssu MC Unggul Tipis 1-0 dari Perseba
Manajemen Persibo Bojonegoro Kembali Ingatkan Elemen Suporter
Di Ambang Degradasi, Persebaya Surabaya Harus Tundukkan Persis Solo
Jamu Persis Solo, Persebaya Surabaya Tanpa Bek
PSMP Mojokerto Kantongi Izin Tuan Rumah Leg Kedua
Puncaki Klasemen Sementara Liga 3 Grup M, Ini Persiapan Perssu MC Hadapi Putaran Kedua
Persepam Berambisi Kembali Kalahkan Persesa Sampang
Gresik United Fokus Benahi Tim Jelang Pertandingan Terakhir Grup 3
Dokter

In [5]:
politik = await scrape_beritajatim('https://beritajatim.com/kanal/politik-pemerintahan/',30,'Politik')

Vaksinasi 89 Persen, Pemkot Blitar Terima Penghargaan Zero Reported Case PMK
Pemkot Blitar Raih 2 Penghargaan Bergengsi dari Kementerian LHK
Pemkot Blitar Komitmen Pertahankan Predikat Swasti Saba Wisatara
Heri Cahyono Resmi Umumkan Niat Maju Pilkada Kota Malang
Gandeng PT Sampoerna, Pemkot Blitar Gelar Sosialisasi Lowongan Kerja Bagi Warga Miskin
Prof Henri Subiakto: Cara Ganjar Atasi Konflik di Papua Tepat
Begini Cara Pemdes Cileng Magetan Tangani Anak Stunting
Diskominfo Kediri Sosialisasi Perwali Guna Perkuat Peran KIM
PDIP Jember: Anies Lawan Bagus bagi Ganjar dalam Debat Capres
PAW DPRD Pamekasan, Afifurrahman Gantikan Mohammad Sahur
Sebanyak 20 TPS di Kabupaten Blitar Termasuk Rawan Bencana dan Terpencil
PAN Jember: Anies Lawan Terberat Prabowo Saat Debat Capres
Bawaslu Bangkalan Tertibkan Ratusan APK Melanggar Aturan
Gereja Mawar Sharon Bagi Ribuan Paket Sembako, Eri Cahyadi: Surabaya Kota Toleransi
Presiden Jokowi Bakal Buka Natal Nasional di Surabaya
Gebyar Undian Lunas PBB, 

In [6]:
hasil1 = pd.DataFrame(pendidikan, columns=['Judul', 'Isi','Tanggal', 'Kategori'])
hasil2 = pd.DataFrame(politik, columns=['Judul', 'Isi', 'Tanggal','Kategori'])
hasil3 = pd.DataFrame(sport, columns=['Judul', 'Isi', 'Tanggal','Kategori'])

In [7]:
combined_df = pd.concat([hasil1, hasil2, hasil3]).reset_index(drop=True)
combined_df

,Judul,Isi,Tanggal,Kategori
0,5 PTN di Indonesia dengan Kelas Internasional,Surabaya (beritajatim.com)– Program kelas inte...,"Kamis, 14 Desember 2023, 02:00 WIB",Pendidikan
1,PPK Ormawa HMPBK Unesa Boyong Tiga Penghargaan...,Surabaya (beritajatim.com) –Program Penguatan ...,"Kamis, 14 Desember 2023, 00:52 WIB",Pendidikan
2,Rektor Unitomo Ajak Mahasiswanya Tak Apatis pa...,Surabaya (beritajatim.com) –Rektor Universitas...,"Rabu, 13 Desember 2023, 23:08 WIB",Pendidikan
3,PWI Gresik dan SMA Sunan Giri Gelar Pelatihan ...,Gresik (beritajatim.com)– PWI (Persatuan Warta...,"Rabu, 13 Desember 2023, 19:48 WIB",Pendidikan
4,Dialog Gusdurian di Untag Surabaya Ajak Perlua...,Surabaya (beritajatim.com)– Universitas 17 Agu...,"Rabu, 13 Desember 2023, 16:55 WIB",Pendidikan
...,...,...,...,...
1095,Sukses Pertandingan Piala Dunia U-17 di Suraba...,Surabaya (beritajatim.com)- Apel gelar pasukan...,"Jumat, 10 November 2023, 14:48 WIB",Sport
1096,Jakarta Sudah Siap Menggelar Piala Dunia U-17 ...,Jakarta (beritajatim.com)– Jakarta sebagai sal...,"Jumat, 10 November 2023, 11:26 WIB",Sport
1097,Kaledonia Baru Siap Kejutkan Para Raksasa Grup C,Jakarta (beritajatim.com)– Piala Dunia U-17 20...,"Jumat, 10 November 2023, 09:44 WIB",Sport
1098,Kanada Siap Beri Kejutan untuk Spanyol di Laga...,Surakarta (beritajatim.com)– Kanada tahu bahwa...,"Jumat, 10 November 2023, 09:34 WIB",Sport


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df = combined_df.sample(frac=1).reset_index(drop=True)
df.to_csv('/content/drive/MyDrive/ppw/beritajatim.csv', index=False)

In [10]:
df

,Judul,Isi,Tanggal,Kategori
0,Pelatih Ungkap Dua Faktor Kekalahan Persema da...,Jember (beritajatim.com)– Persema Malang dihaj...,"Selasa, 5 Desember 2023, 21:52 WIB",Sport
1,"Jumpa Persebaya, Persis Belum Tentu Dipimpin L...",Jakarta (beritajatim.com)– Persis Solo belum t...,"Senin, 11 Desember 2023, 09:56 WIB",Sport
2,Eits! 229 Orang di Jombang Positif HIV/AIDS,Jombang (beritajatim.com) –Sebanyak 229 orang ...,"Minggu, 3 Desember 2023, 07:32 WIB",Pendidikan
3,Bupati Sidoarjo Siagakan BPBD Antisipasi Benca...,Sidoarjo (beritajatim.com)– Bupati Sidoarjo H....,"Rabu, 13 Desember 2023, 14:38 WIB",Politik
4,"Pesta Siaga ASIK 2023, Bentuk Generasi Berkara...",Lamongan (beritajatim.com) –Pramuka Lamongan m...,"Selasa, 21 November 2023, 21:54 WIB",Pendidikan
...,...,...,...,...
1095,PKS Tuban Gelar Kampanye Perdana dengan Flashmob,Tuban (beritajatim.com) –DPD Partai Keadilan S...,"Minggu, 10 Desember 2023, 18:46 WIB",Politik
1096,"Musim Pancaroba, Peternak Kediri Waspada Penya...",Kediri (beritajatim.com) –Memasuki musim panca...,"Selasa, 14 November 2023, 09:10 WIB",Pendidikan
1097,"Dukung Ganjar, Pemulung di Blitar Ikut Nobar D...",Blitar (beritajatim.com) –Seorang pemulung di ...,"Selasa, 12 Desember 2023, 21:48 WIB",Politik
1098,"Lawan Simo Putra, PSMP Target Kembali Raih Poin",Mojokerto (beritajatim.com)– Tim PS Mojokerto ...,"Kamis, 7 Desember 2023, 11:15 WIB",Sport


In [11]:
pd.crosstab(index=df['Kategori'], columns='count')

col_0,count
Kategori,
Pendidikan,400
Politik,300
Sport,400
